# Training_Nueral_Network
### Vanila Gradient Descent (실제 기울기)
- Gradient를 한 번 업데이트 하기 위해 모든 학습 데이터를 사용
    > 10만개 이상의 데이터를 계산하기에는 비효율적임.
<br>
### Stochastic Gradient Descent (확률적 기울기)
- Gradient를 한 번 업데이트 하기 위해 일부의 데이터만을 사용
    > 전체 데이터 셋에서 Batch_size 대로 잘라서 그것에 대해 손실 함수, Gradient를 내어 계산.

- 학습 데이터 전체를 한 번 학습하는 것을 Epoch, 한 번 Gradient를 구하는 단위를 Batch라고 한다.

- Vanila Gradient Descent가 Stochastic Gradient Descent가 더 좋은 결과가 나타난다.
    > 그럼에도 Stochastic Gradient Descent를 사용하는 이유는?<br>

    > Gradient를 한 번 Update하는데 걸리는 시간이 수백, 수천배 이상 차이가 남.<br>
    > SGD에서 Gradient의 질이 떨어지는 부분은 심화 최적화 알고리즘으로 커버 가능.

# 매개 변수(Parameters)
### 학습 가능한 매개 변수(Trainable Parameters)
- 최적화 방법 또는 다른 방법으로 학습되는 변수
### 초매개변수(Hyper-parameter)
- 알고리즘 결과에 영향을 주지만, 처음 결정한 후 학습되지 않는 변수
    - 최적화 기법
    - 전결합 계층
매개 변수(Parameters)
### 학습 가능한 매개 변수(Trainable Parameters)
- 최적화 방법 또는 다른 방법으로 학습되는 변수 <br>
### 초매개변수(Hyper-parameter)
- 알고리즘 결과에 영향을 주지만, 처음 결정한 후 학습되지 않는 변수 <br>

    - 최적화 기법
        > 알고리즘의 종류, 학습률, 모멘텀 계수, RMSProp계수 등
    - 전결합 계층(Fully-Connected-Layer)
        > 뉴런의 개수, Weight 초기화 방법, Bias 초기값, 활성 함수의 선택
    - 미니배치 학습법
        > 배치 크기, 총 Epoch 수, 손실 함수의 선택, 학습/검증 데이터셋 분류 방식

# 뉴럴 네트워크 학습 알고리즘 구현

### import modules

In [5]:
import tensorflow as tf

### 하이퍼파라미터 정의

In [22]:
EPOCHS = 10

### 네트워크 구조 정의

In [23]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__() # tf.keras.Model의 init 함수를 먼저 call 해줌.
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        # 28 x 28 영상형태의 MNIST를 Vector 형태로 Flatten 해줌.
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
# self.dense5는 출력함수, 10개를 출력할 것이며, Multiclass_classification을 위해서는 'softmax' activation을 사용해야한다.
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

### 학습 함수 구현

In [24]:
@tf.function # 사용하게 되면 일반적인 python 함수가 아니라 아래 연산들이 모두 tf 연산으로 바뀜. 더 최적화된 형태로 compile 돼서 돌아감.
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape: # 안에서 일어나는 모든 연산에 대해 Gradient를 계산
        predictions = model(images) # Batchsize x 10(Classes) 
        loss = loss_object(labels, predictions) # 여기까지 Forward 연산
    gradients = tape.gradient(loss, model.trainable_variables)
    # tape에 저장된 것을 이용해 gradients 계산. loss를 각각 trainable_varibles 미분한 것을 gradients로 받아주는 것
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # gradient랑 trainable_varibles를 같이 묶어 apply_gradients를 통해 optimizer에 update 헤주는 것.
    train_loss(loss)
    train_accuracy(labels, predictions)

### 테스트 함수 구현

In [25]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images) # Batchsize x 10(Classes) 
    loss = loss_object(labels, predictions) # 여기까지 Forward 연산

    test_loss(loss)
    test_accuracy(labels, predictions)

### 데이터 불러오기

In [26]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# 0~255의 숫자로 저장이 되어있기에 255로 나누어 float 형태로 바꾸어줘야함.

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 모델 생성

In [27]:
model = MyModel()

### 손실함수 및 최적화 알고리즘 정의

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# loss_object는 label과 prediction의 입력을 받게 됨.
# prediction : BatchSize x Classes
optimizer = tf.keras.optimizers.Adam()

### 성능 지표 정의

In [29]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 학습 루프 구현

In [30]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss : {}, Accuracy: {}, Test Loss : {}, Test Accuracy: {}'
    print(template.format(epoch +1,
    train_loss.result(),
    train_accuracy.result() * 100,
    test_loss.result(),
    test_accuracy.result() * 100))
# 누적되지 않게끔 reset_states를 통해 매번 초기화 해줌
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss : 0.27360546588897705, Accuracy: 91.8316650390625, Test Loss : 0.17664393782615662, Test Accuracy: 94.47000122070312
Epoch 2, Loss : 0.13846799731254578, Accuracy: 95.78166961669922, Test Loss : 0.1769893318414688, Test Accuracy: 94.66999816894531
Epoch 3, Loss : 0.10860887914896011, Accuracy: 96.69999694824219, Test Loss : 0.12071310728788376, Test Accuracy: 96.10000610351562
Epoch 4, Loss : 0.08931570500135422, Accuracy: 97.23500061035156, Test Loss : 0.16584593057632446, Test Accuracy: 95.38999938964844
Epoch 5, Loss : 0.07730986922979355, Accuracy: 97.625, Test Loss : 0.11720754951238632, Test Accuracy: 96.45000457763672
Epoch 6, Loss : 0.06961990892887115, Accuracy: 97.82166290283203, Test Loss : 0.1280515044927597, Test Accuracy: 96.4000015258789
Epoch 7, Loss : 0.060718003660440445, Accuracy: 98.05833435058594, Test Loss : 0.11466178297996521, Test Accuracy: 96.80000305175781
Epoch 8, Loss : 0.05548420175909996, Accuracy: 98.25, Test Loss : 0.11317700147628784, Tes